In [7]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster
# import asyncio

In [8]:
start_date = "2010-01-01"
end_date = "2010-12-31"
comids = [2043493]

nwm = NWM(start_date=start_date, end_date=end_date, comids=comids)
scheduler = LocalCluster(n_workers=5, threads_per_worker=4, processes=False)
scheduler

C:\Users\dsmith\Anaconda3\envs\hms_kube\lib\site-packages\distributed\node.py:181: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57545 instead
  warnings.warn(


LocalCluster(4186cb9c, 'inproc://10.144.72.139/34768/19', workers=5, threads=20, memory=127.76 GiB)

In [ ]:
%%time
# %%prun
nwm.request_timeseries(scheduler=scheduler, optimize=True, nwm_21=True)
nwm.data

In [ ]:
%%time
df = nwm.set_output(return_dataframe=False)
nwm.output

In [ ]:
%%time
nwm.request_timeseries(scheduler=scheduler, optimize=False, nwm_2=True)
nwm.data

In [ ]:
%%time
nwm.set_output()
nwm.output

In [9]:
import xarray as xr
import fsspec
import dask
import s3fs

In [10]:
nwm_url = "s3://noaa-nwm-retrospective-2-1-pds"

In [ ]:
%%time
drop_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral"]
ds = xr.open_zarr(fsspec.get_mapper(nwm_url, anon=True), consolidated=True, drop_variables=drop_variables)
# ds = ds.chunk(chunks='auto')

In [15]:
%%time
zarr_file = nwm_url
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=zarr_file, s3=s3, check=False)
ds_nwm_chrtout = xr.open_zarr(store=store, consolidated=True)
ds_nwm_chrtout

KeyError: '.zmetadata'

In [ ]:
%%time
ds_nwm_feature = ds_nwm_chrtout.isel(feature_id==f'{self.comids}'.encode())
streamflow_nwm = ds_nwm_feature.streamflow.load()
streamflow_nwm_df = streamflow_nwm.squeeze('feature_id').to_dataframe()
streamflow_nwm_df

In [ ]:
%%time
variables = ["streamflow", "velocity"]
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
# with dask.config.set(**{'array.slicing.split_large_chunks': True}):
ds_data = ds[variables].sel(feature_id=nwm.comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
ds_data = ds.sel(feature_id=nwm.comids)
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
ds_data

In [ ]:
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
print(f"Start: {start_string}, End: {end_string}")
ds_data

In [ ]:
all_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral", "streamflow", "velocity"]
d_v = list(set(all_variables) - set(variables))
d_v